In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
import time
import imp
import os
import os, os.path
import datetime as dt
import dateutil.parser as dp
import dill

%matplotlib inline

In [2]:
data_directory = "C:/Users/YuriTurygin/Desktop/NN-PPM/new data"

# Create lists of data files (3 groups: regular attributes, GEO, and Seller composition)
pools_attributes_files = list()
for root, dirs, files in os.walk(data_directory + "/pools attributes"):
    for name in files:
        pools_attributes_files.append(os.path.join(root, name))        
        
pools_geo_pct_files = list()
for root, dirs, files in os.walk(data_directory + "/geo pct"):
    for name in files:
        pools_geo_pct_files.append(os.path.join(root, name))  
        
pools_seller_pct_files = list()
for root, dirs, files in os.walk(data_directory + "/seller pct"):
    for name in files:
        pools_seller_pct_files.append(os.path.join(root, name)) 
        
print("# of attr files: ",len(pools_attributes_files))
print("# of geo pct files: ",len(pools_geo_pct_files))
print("# of seller pct files: ",len(pools_seller_pct_files))

# of attr files:  130
# of geo pct files:  130
# of seller pct files:  130


In [3]:
# Read in all files with regular pool attributes
attr_df = pd.DataFrame()
for file in pools_attributes_files:
    one_month_issue_pools = pd.read_csv(file)
    if len(attr_df)==0:
        attr_df = one_month_issue_pools
    else:
        # df1.columns.difference(df2.columns)
        # df2.columns.difference(df1.columns)
        if all(attr_df.columns == one_month_issue_pools.columns):
            attr_df = pd.concat([attr_df,one_month_issue_pools])
        else:
            print(f'columns missmatch for {file}')

    #print('Done loading ' + file)

C:\Users\YuriTurygin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\YuriTurygin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\YuriTurygin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\YuriTurygin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result

In [4]:
attr_df.columns

Index(['poolno', 'asofdate', 'cusip', 'Prefix', 'spread', 'Cpr1', 'Cpr3',
       'Cpr6', 'Cpr12', 'Cpr24', 'CprLife', 'SMM', 'DayCount', 'OBal', 'CBal',
       'pbal', 'paydown', 'Prepay', 'factor', 'OCoupon', 'Coupon', 'OWac',
       'Wac', 'Wam', 'Age', 'aols', 'waols', 'ONLoans', 'cnloans', 'PCNLoans',
       'PPNLoans', 'OSato', 'CSato', 'oltv', 'cltv', '%cltv_80', '%cltv_105',
       '%cltv_125', '%ccltv_80', '%ccltv_105', '%ccltv_125', 'fico',
       '%FedHold', '%CMOHold', 'dti', 'codti', '%CashWindow', '%Majors',
       'ocltv', 'ccltv', 'PurpPct_purchase', 'PurpPct_refi',
       'PctChannel_Broker', 'PctChannel_Corr', 'PctChannel_Retail',
       'OccPct_investor', 'OccPct_owner', 'PropUnitsPct_2-4', 'Label'],
      dtype='object')

In [5]:
print(f'attr_df.shape = {attr_df.shape}')

attr_df.shape = (2578718, 59)


In [6]:
# Read in all files with pools GEO info
geo_df = pd.DataFrame()
for file in pools_geo_pct_files:
    one_month_issue_pools = pd.read_csv(file)
    if len(geo_df)==0:
        geo_df = one_month_issue_pools
    else:
        # df1.columns.difference(df2.columns)
        # df2.columns.difference(df1.columns)
        if all(geo_df.columns == one_month_issue_pools.columns):
            geo_df = pd.concat([geo_df,one_month_issue_pools])
        else:
            print(f'columns missmatch for {file}')

    #print('Done loading ' + file)

In [7]:
geo_df.columns

Index(['poolno', 'asofdate', 'cusip', 'StatePct_AK', 'StatePct_AL',
       'StatePct_AR', 'StatePct_AZ', 'StatePct_CA', 'StatePct_CO',
       'StatePct_CT', 'StatePct_DC', 'StatePct_DE', 'StatePct_FL',
       'StatePct_GA', 'StatePct_GU', 'StatePct_HI', 'StatePct_IA',
       'StatePct_ID', 'StatePct_IL', 'StatePct_IN', 'StatePct_KS',
       'StatePct_KY', 'StatePct_LA', 'StatePct_MA', 'StatePct_MD',
       'StatePct_ME', 'StatePct_MI', 'StatePct_MN', 'StatePct_MO',
       'StatePct_MS', 'StatePct_MT', 'StatePct_NC', 'StatePct_ND',
       'StatePct_NE', 'StatePct_NH', 'StatePct_NJ', 'StatePct_NM',
       'StatePct_NV', 'StatePct_NY', 'StatePct_OH', 'StatePct_OK',
       'StatePct_OR', 'StatePct_PA', 'StatePct_PR', 'StatePct_RI',
       'StatePct_SC', 'StatePct_SD', 'StatePct_TN', 'StatePct_TX',
       'StatePct_UT', 'StatePct_VA', 'StatePct_VI', 'StatePct_VT',
       'StatePct_WA', 'StatePct_WI', 'StatePct_WV', 'StatePct_WY', 'Label'],
      dtype='object')

In [8]:
print(f'geo_df.shape = {geo_df.shape}')

geo_df.shape = (1110386, 58)


In [9]:
# Read in all files with pools Seller info
seller_df = pd.DataFrame()
for file in pools_seller_pct_files:
    one_month_issue_pools = pd.read_csv(file)
    if len(seller_df)==0:
        seller_df = one_month_issue_pools
    else:
        # df1.columns.difference(df2.columns)
        # df2.columns.difference(df1.columns)
        if all(seller_df.columns == one_month_issue_pools.columns):
            seller_df = pd.concat([seller_df,one_month_issue_pools])
        else:
            print(f'columns missmatch for {file}')
    #print('Done loading ' + file)

In [10]:
seller_df.columns

Index(['poolno', 'asofdate', 'SellerPct_AMRHT', 'SellerPct_ALS',
       'SellerPct_CAFULL', 'SellerPct_CNTL', 'SellerPct_CITIZ', 'SellerPct_53',
       'SellerPct_FIR', 'SellerPct_FRDOM', 'SellerPct_GUILD',
       'SellerPct_CHASE', 'SellerPct_LLSL', 'SellerPct_MATRX', 'SellerPct_NCM',
       'SellerPct_NATIONST', 'SellerPct_NRESM', 'SellerPct_PNYMAC',
       'SellerPct_PILOSI', 'SellerPct_QUICK', 'SellerPct_REG',
       'SellerPct_RMSC', 'SellerPct_UNSHFI', 'SellerPct_WFHM', 'cusip',
       'Prefix', 'Label'],
      dtype='object')

In [11]:
print(f'seller_df.shape = {seller_df.shape}')

seller_df.shape = (1486443, 27)


In [12]:
print(f'attr_df.shape = {attr_df.shape}')

attr_df.shape = (2578718, 59)


In [13]:
attr_df.drop(['Prefix','Cpr1','Cpr3','Cpr6','Cpr12','Cpr24','CprLife','pbal','paydown','Prepay','OCoupon','OWac','PCNLoans','PPNLoans','OSato','%cltv_80','%cltv_105','%cltv_125','%ccltv_80','%ccltv_105','%ccltv_125','%FedHold','%CMOHold','dti','codti'],axis=1,inplace=True)

In [14]:
attr_df.dropna(subset=['CBal','SMM','Wac','aols','fico','OccPct_investor','OccPct_owner','PropUnitsPct_2-4'],inplace=True)

In [15]:
attr_df.sort_values(by=['poolno','asofdate'],inplace=True)

In [16]:
attr_df.fillna(method='ffill',inplace=True)

In [17]:
attr_df['CBal'] = attr_df['CBal'].apply(lambda x: float(x))

In [18]:
attr_df.rename(columns={'Label':'pool_issue_month'},inplace=True)

In [19]:
attr_df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2574122 entries, 12849 to 21251
Data columns (total 34 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   poolno             2574122 non-null  object 
 1   asofdate           2574122 non-null  int64  
 2   cusip              2574122 non-null  object 
 3   spread             2574122 non-null  object 
 4   SMM                2574122 non-null  float64
 5   DayCount           2574122 non-null  float64
 6   OBal               2574122 non-null  float64
 7   CBal               2574122 non-null  float64
 8   factor             2574122 non-null  float64
 9   Coupon             2574122 non-null  float64
 10  Wac                2574122 non-null  float64
 11  Wam                2574122 non-null  float64
 12  Age                2574122 non-null  float64
 13  aols               2574122 non-null  float64
 14  waols              2574122 non-null  object 
 15  ONLoans            2574122 non

In [20]:
geo_df.drop('Label',axis=1,inplace=True)

In [21]:
geo_df.sort_values(by=['poolno','asofdate'],inplace=True)

In [22]:
geo_df.fillna(method='ffill',inplace=True)

In [23]:
geo_df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1110386 entries, 1638 to 4436
Data columns (total 57 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   poolno       1110386 non-null  object 
 1   asofdate     1110386 non-null  int64  
 2   cusip        1110386 non-null  object 
 3   StatePct_AK  1110386 non-null  float64
 4   StatePct_AL  1110386 non-null  object 
 5   StatePct_AR  1110386 non-null  float64
 6   StatePct_AZ  1110386 non-null  float64
 7   StatePct_CA  1110386 non-null  float64
 8   StatePct_CO  1110386 non-null  object 
 9   StatePct_CT  1110386 non-null  float64
 10  StatePct_DC  1110386 non-null  float64
 11  StatePct_DE  1110386 non-null  float64
 12  StatePct_FL  1110386 non-null  float64
 13  StatePct_GA  1110386 non-null  float64
 14  StatePct_GU  1110386 non-null  float64
 15  StatePct_HI  1110386 non-null  float64
 16  StatePct_IA  1110386 non-null  float64
 17  StatePct_ID  1110386 non-null  float64
 18  St

In [24]:
geo_columns = ['StatePct_AK','StatePct_AL','StatePct_AR','StatePct_AZ','StatePct_CA','StatePct_CO','StatePct_CT',
                'StatePct_DC','StatePct_DE','StatePct_FL','StatePct_GA','StatePct_GU','StatePct_HI','StatePct_IA',
                'StatePct_ID','StatePct_IL','StatePct_IN','StatePct_KS','StatePct_KY','StatePct_LA','StatePct_MA',
                'StatePct_MD','StatePct_ME','StatePct_MI','StatePct_MN','StatePct_MO','StatePct_MS','StatePct_MT',
                'StatePct_NC','StatePct_ND','StatePct_NE','StatePct_NH','StatePct_NJ','StatePct_NM','StatePct_NV',
                'StatePct_NY','StatePct_OH','StatePct_OK','StatePct_OR','StatePct_PA','StatePct_PR','StatePct_RI',
                'StatePct_SC','StatePct_SD','StatePct_TN','StatePct_TX','StatePct_UT','StatePct_VA','StatePct_VI',
                'StatePct_VT','StatePct_WA','StatePct_WI','StatePct_WV','StatePct_WY']

In [25]:
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

In [26]:
geo_df[geo_columns] = geo_df[geo_columns].applymap(lambda x: float(x) if isfloat(x) else np.nan)

In [27]:
geo_df.fillna(0,inplace=True)

In [28]:
geo_df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1110386 entries, 1638 to 4436
Data columns (total 57 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   poolno       1110386 non-null  object 
 1   asofdate     1110386 non-null  int64  
 2   cusip        1110386 non-null  object 
 3   StatePct_AK  1110386 non-null  float64
 4   StatePct_AL  1110386 non-null  float64
 5   StatePct_AR  1110386 non-null  float64
 6   StatePct_AZ  1110386 non-null  float64
 7   StatePct_CA  1110386 non-null  float64
 8   StatePct_CO  1110386 non-null  float64
 9   StatePct_CT  1110386 non-null  float64
 10  StatePct_DC  1110386 non-null  float64
 11  StatePct_DE  1110386 non-null  float64
 12  StatePct_FL  1110386 non-null  float64
 13  StatePct_GA  1110386 non-null  float64
 14  StatePct_GU  1110386 non-null  float64
 15  StatePct_HI  1110386 non-null  float64
 16  StatePct_IA  1110386 non-null  float64
 17  StatePct_ID  1110386 non-null  float64
 18  St

In [29]:
seller_df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1486443 entries, 0 to 1657
Data columns (total 27 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   poolno              1486443 non-null  object 
 1   asofdate            1486443 non-null  int64  
 2   SellerPct_AMRHT     1485051 non-null  float64
 3   SellerPct_ALS       1485051 non-null  float64
 4   SellerPct_CAFULL    1485051 non-null  float64
 5   SellerPct_CNTL      1485051 non-null  float64
 6   SellerPct_CITIZ     1485051 non-null  float64
 7   SellerPct_53        1485051 non-null  float64
 8   SellerPct_FIR       1485051 non-null  float64
 9   SellerPct_FRDOM     1485051 non-null  float64
 10  SellerPct_GUILD     1485051 non-null  float64
 11  SellerPct_CHASE     1485050 non-null  float64
 12  SellerPct_LLSL      1485050 non-null  float64
 13  SellerPct_MATRX     1485050 non-null  float64
 14  SellerPct_NCM       1485050 non-null  float64
 15  SellerPct_NATIONST

In [30]:
seller_df.drop(['Label','Prefix'],axis=1,inplace=True)

In [31]:
seller_df.sort_values(by=['poolno','asofdate'],inplace=True)

In [32]:
seller_df.fillna(method='ffill',inplace=True)

In [33]:
seller_columns = ['SellerPct_AMRHT','SellerPct_ALS','SellerPct_CAFULL','SellerPct_CNTL','SellerPct_CITIZ',
                  'SellerPct_53','SellerPct_FIR','SellerPct_FRDOM','SellerPct_GUILD','SellerPct_CHASE',
                  'SellerPct_LLSL','SellerPct_MATRX','SellerPct_NCM','SellerPct_NATIONST','SellerPct_NRESM',
                  'SellerPct_PNYMAC','SellerPct_PILOSI','SellerPct_QUICK','SellerPct_REG','SellerPct_RMSC',
                  'SellerPct_UNSHFI','SellerPct_WFHM']

In [34]:
seller_df[seller_columns] = seller_df[seller_columns].applymap(lambda x: float(x) if isfloat(x) else np.nan)

In [35]:
seller_df.fillna(0,inplace=True)

In [36]:
seller_df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1486443 entries, 12849 to 4436
Data columns (total 25 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   poolno              1486443 non-null  object 
 1   asofdate            1486443 non-null  int64  
 2   SellerPct_AMRHT     1486443 non-null  float64
 3   SellerPct_ALS       1486443 non-null  float64
 4   SellerPct_CAFULL    1486443 non-null  float64
 5   SellerPct_CNTL      1486443 non-null  float64
 6   SellerPct_CITIZ     1486443 non-null  float64
 7   SellerPct_53        1486443 non-null  float64
 8   SellerPct_FIR       1486443 non-null  float64
 9   SellerPct_FRDOM     1486443 non-null  float64
 10  SellerPct_GUILD     1486443 non-null  float64
 11  SellerPct_CHASE     1486443 non-null  float64
 12  SellerPct_LLSL      1486443 non-null  float64
 13  SellerPct_MATRX     1486443 non-null  float64
 14  SellerPct_NCM       1486443 non-null  float64
 15  SellerPct_NATI

In [37]:
dill.dump_session('notebook_env_all_3_dfs.db')

-----------------------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------------------
Section above performed data loading from raw csv files, performed data cleaning and filled in NAs.
Section below combines the three DataFrames (attr_df, geo_df, seller_df) into a single DataFrame called df
We then save df into a dill session memory file and csv files for pools with >=250 current number of loans
Due to memory issues we only keep a subset of df consisting only of pools with >=250 current number of loans 

-----------------------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------------------

In [38]:
#dill.load_session('notebook_env_all_3_dfs.db')

In [39]:
print(f'attr_df.shape   = {attr_df.shape}')
print(f'geo_df.shape    = {geo_df.shape}')
print(f'seller_df.shape = {seller_df.shape}')

attr_df.shape   = (2574122, 34)
geo_df.shape    = (1110386, 57)
seller_df.shape = (1486443, 25)


In [40]:
nloans = 250
attr_df = attr_df[attr_df['cnloans']>=nloans]

print(f'attr_df.shape = {attr_df.shape}')

attr_df.shape = (109186, 34)


In [41]:
geo_df.drop('poolno',axis=1,inplace=True)

In [42]:
df = pd.merge(attr_df,geo_df,on=['cusip','asofdate'],how='left')

In [43]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109186 entries, 0 to 109185
Data columns (total 88 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   poolno             109186 non-null  object 
 1   asofdate           109186 non-null  int64  
 2   cusip              109186 non-null  object 
 3   spread             109186 non-null  object 
 4   SMM                109186 non-null  float64
 5   DayCount           109186 non-null  float64
 6   OBal               109186 non-null  float64
 7   CBal               109186 non-null  float64
 8   factor             109186 non-null  float64
 9   Coupon             109186 non-null  float64
 10  Wac                109186 non-null  float64
 11  Wam                109186 non-null  float64
 12  Age                109186 non-null  float64
 13  aols               109186 non-null  float64
 14  waols              109186 non-null  object 
 15  ONLoans            109186 non-null  float64
 16  cn

In [44]:
seller_df.drop('poolno',axis=1,inplace=True)

In [45]:
df = pd.merge(df,seller_df,on=['cusip','asofdate'],how='left')

In [46]:
df.fillna(0,inplace=True)

In [47]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109186 entries, 0 to 109185
Data columns (total 110 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   poolno              109186 non-null  object 
 1   asofdate            109186 non-null  int64  
 2   cusip               109186 non-null  object 
 3   spread              109186 non-null  object 
 4   SMM                 109186 non-null  float64
 5   DayCount            109186 non-null  float64
 6   OBal                109186 non-null  float64
 7   CBal                109186 non-null  float64
 8   factor              109186 non-null  float64
 9   Coupon              109186 non-null  float64
 10  Wac                 109186 non-null  float64
 11  Wam                 109186 non-null  float64
 12  Age                 109186 non-null  float64
 13  aols                109186 non-null  float64
 14  waols               109186 non-null  object 
 15  ONLoans             109186 non-nu

In [48]:
dill.dump_session('notebook_env__df.db')

In [49]:
# Write df into a sequence of small files
nloans = 250
ddf = df[df['cnloans']>=nloans]

print(f'ddf.shape = {ddf.shape}')

rows = ddf.shape[0]

N = rows//30000 + 1

for i in range(N):
    ddf.iloc[i*30000:(i+1)*30000,].to_csv(f'new data/pools_ge{nloans}_loans_part{i+1}.csv',index=False)

ddf.shape = (109186, 110)
